In [ ]:
import pandas as pd
import numpy as np
from ast import literal_eval
import copy
import itertools
from tqdm import tqdm
from textblob import TextBlob
import nltk
nltk.download('punkt')
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
import networkx as nx
import matplotlib.pyplot as plt
from scipy import spatial
%matplotlib inline
import math
import random 
from scipy.spatial import distance
import time
import random
import pickle as pk
import multiprocessing
import ast
from itertools import permutations 

In [ ]:
Univ2_data=pd.read_csv('Univ2_dataset.csv',encoding = "ISO-8859-1")

In [ ]:
Univ2_data.info()

In [ ]:
#removing two not required rows
Univ2_data.drop([20,23],inplace=True)

In [ ]:
#removing all the department number starting with 1
Univ2_data.drop(Univ2_data[Univ2_data['DepartmentNumber'].str.contains('^1')].index, inplace = True) 

In [ ]:
#checking for the department number if all the numbers starting with 1 are not present
Univ2_data['DepartmentNumber'].str.contains('^1', case=False)

In [ ]:
#removing the duplicates rows based on course number
Univ2_data.drop_duplicates(subset=['course_number'],inplace=True)

In [ ]:
#to reset the index
Univ2_data.reset_index(drop=True,inplace=True)

In [ ]:
Univ2_data.head()

In [ ]:
Univ2_data[Univ2_data['course_number']=='STATS 306B'].index

In [ ]:
#creating the six lists with courses denoting six different areas required to complete the degree
data_science_math_list=['STATS 200','STATS 300A','STATS 203','STATS 305A','STATS 315A','CME 302','CME 308']
data_science_experimentative_elective=['STATS 263','ECON 271','MS&E 327']
data_science_software_list=['CME 212','CME 213','CME 305','CME 307','CME 323','CME 364A','CS 246']
data_science_machine_learning_list=['STATS 231','STATS 315B','CS 221','CS 224N','CS 230','CS 231N','CS 234','CS 236']
data_science_practical_list=['STATS 299','STATS 390']
data_science_elective_list=['STATS 306B','CS 228','CS 229','CME 211','MATH 21','STATS 305','ECON 270',
 'CME 200','STATS 204']

In [ ]:
data_science_core_list=data_science_math_list + data_science_experimentative_elective + data_science_software_list + data_science_machine_learning_list + data_science_practical_list

In [ ]:
len(data_science_core_list)

In [ ]:
#creating the prereq dict by checking from univ-2 website
prereq_dict_3={'CME 212': [['CME 200', 'CME 211']],
 'CME 213': [],
 'CME 302': [],
 'CME 305': [],
 'CME 308': [],
 'CME 323': [],
 'CME 364A': [],
 'CS 221': [],
 'CS 224N': ['CS 221', 'CS 229'],
 'CS 230': [],
 'CS 231N': [['CS 229', 'MATH 21']],
 'CS 234': ['CS 229'],
 'CS 236': ['CS 221', 'CS 228', 'CS 229', 'CS 230'],
 'CS 246': [],
 'ECON 271': ['ECON 270'],
 'MS&E 327': [],
 'STATS 200': [],
 'STATS 203': [],
 'STATS 204': [],
 'STATS 231': [],
 'STATS 299': [],
 'STATS 305A': [],
 'STATS 315A': ['STATS 305','STATS 306B'],
 'STATS 315B': [],
 'STATS 390': []}

In [ ]:
#Converting the format of title column in the dataset to string format
Univ2_data['Title']=Univ2_data['Title'].apply(str)

#cleansing of data
def clean_text(text):
    #Convert to lowercase to maintain consistency
    text = text.lower()
    return text

Univ2_data['Title']=Univ2_data['Title'].apply(clean_text)

In [ ]:
#Converting the format of title column in the dataset to string format
def getting_nouns(text):
  text_blob_object=TextBlob(text)

  return text_blob_object.words


Univ2_data['words']=Univ2_data['Title'].apply(getting_nouns)

In [ ]:
#removing stopwords in nltk library


Univ2_data['words'] = Univ2_data['words'].apply(lambda x:[item for item in x if item not in stop])
#converting words column from the dataset into a single list
list_of_topics=Univ2_data['words'].tolist()
#converting into a single list
merged = list(itertools.chain.from_iterable(list_of_topics))
#removing the duplicates from the list
top_100_topics=list(dict.fromkeys(merged))

In [ ]:
print(*top_100_topics,sep=",")

In [ ]:
#making units dictionary to track the course and its numbers of units 
course_units_dict={}
for i in range(len(Univ2_data['course_number'])):
  course_units_dict[Univ2_data['course_number'][i]]=Univ2_data['UnitsMin'][i]

course_units_dict['ECON 271']=3
course_units_dict['STATS 300A']=3
course_units_dict['STATS 390']=1
course_units_dict['STATS 299']=2

In [ ]:
#Update function for feature vector update at a node

def update_feature_vector(feature_vector,topic_vector,course_list):
  for i in course_list[0]:
    for j in range(len(topic_vector)):
      if i==topic_vector[j]:
         feature_vector[j]=1
  return feature_vector 

#Creating the feature dictionary with course as key and course topics as values
feature_ds_dict={}
course_list_ds=list(Univ2_data['course_number'])
for i in tqdm(course_list_ds):
  temp = list(Univ2_data[Univ2_data['course_number']== i ]['words'])
  feature_vector_ds =[0]*len(top_100_topics)
  feature_vector_ds = update_feature_vector(feature_vector_ds,top_100_topics,temp)
  feature_ds_dict[i]=feature_vector_ds

In [ ]:
#printing the dictionary
for i,j in feature_ds_dict.items():
  print('{} : {}'.format(i,j))

In [ ]:
#to compile one dictinary to distinguish between six different areas.
data_science_dict={}
for i in range(len(course_list_ds)):
  key=course_list_ds[i]
  if key in data_science_math_list:
    data_science_dict[key]=[1,0,0,0,0,0]
  elif key in data_science_experimentative_elective:
    data_science_dict[key]=[0,1,0,0,0,0]
  elif key in data_science_software_list:
    data_science_dict[key]=[0,0,1,0,0,0]
  elif key in data_science_machine_learning_list:
    data_science_dict[key]=[0,0,0,1,0,0]
  elif key in data_science_practical_list:
    data_science_dict[key]=[0,0,0,0,1,0]
  else:
    data_science_dict[key]=[0,0,0,0,0,1]

In [ ]:
#creating prereq dictionary where course is key and prereqs are values
def prereq_dict(data_dict,course_list):
  course_dict={}
  for i in range(len(course_list)):
    key=course_list[i]
    if key in data_dict:
      course_dict[key]=data_dict.get(key)
    else:
      course_dict[key]=[]
  return course_dict

In [ ]:
data_science_prereq_dict=prereq_dict(prereq_dict_3,course_list_ds)

In [ ]:
data_science_prereq_dict['STATS 305A']=['STATS 200']

In [ ]:
edges=[]
for course in course_list_ds:
  #print(course)
  if (data_science_prereq_dict[course]==[]):
    CourseId=course_list_ds[:]
    CourseId.remove(course)
    #print(CourseId)
    for i in CourseId:
      # print(course , (i,course))
      edges.append((i,course))
  else:
      if (len(data_science_prereq_dict[course])!=1 or type(data_science_prereq_dict[course][0])==str):
        for j in data_science_prereq_dict[course]:
          if(j== [] or j==course):
            continue
          else:
            # print(course , (j,course))
            edges.append((j, course)) 
      else:
        l = list(permutations(data_science_prereq_dict[course][0])) 
        for k in l:
          m=0
          while (m in range(len(k)-1)):
            if ((k[m],k[m+1]) in edges)==False:
              edges.append((k[m],k[m+1]))
            m+=1
          if ((k[len(k)-1],course) in edges)==False:   
            edges.append((k[len(k)-1],course))



for course in course_list_ds:
  if(data_science_prereq_dict[course]==[]):
    edges.append(('initial' , course))
# print(CourseId)
# print(edges)
       
G = nx.DiGraph()
G.add_edges_from(edges) 
pos = nx.spring_layout(G)
nx.draw_networkx_nodes(G,pos, cmap=plt.get_cmap('jet'),node_size = 1000,node_color="lightblue")
nx.draw_networkx_edges(G,pos,edge_color='b', edge_cmap=plt.cm.Blues,arrows=True,arrowstyle="->",arrowsize=10)
nx.draw_networkx_labels(G,pos)

plt.show()
#pylab.show()
print(len(edges))

In [ ]:
#creating a movie id dictionary with key as course and node number as value
movies = list(G.nodes())
movie_ids={}
for i,node in enumerate(G.nodes()):
  movie_ids[node]=i

In [ ]:
#creating the topic dictonary with key as course name and topic as value
topic_dict={}
course_topics=list(Univ2_data['words'])
for i in range(len(course_list_ds)):
  topic_dict[course_list_ds[i]]=course_topics[i]

In [ ]:
topic_dict

In [ ]:
#creating input sequence for omega

a=random.sample(data_science_core_list,12)
b=random.sample(data_science_elective_list,3)
def shuffle(a,b):
  list_of_sequences=[]
  sequence_list=[]
  i=0
  while i<1000:
    sequence_list=[i] + list(a + b)
    list_of_sequences.append(sequence_list)
    #to randomly select 12 core courses 
    a=random.sample(a,12)
    #to randonly select the 3 courses from  the elective list
    b=random.sample(data_science_elective_list,3)
    sequence_list=[]
    i=i+1
  return list_of_sequences

In [ ]:
list_sequences=shuffle(a,b)

In [ ]:
list_sequences

In [ ]:
# split the sequence of a user into two sequences one to add to the graph and the other to use it as ground truth for evaluation
def divide_test(test_user_sequence):
    user_id = test_user_sequence[0]
    items_in_sequence = test_user_sequence[1:]
    l = len(items_in_sequence)//2
    previous = items_in_sequence[:l]

    left = items_in_sequence[l:]

    return user_id , previous , left

def compute_edges(gra, sequence):
    #start_time = time.time()
    SG= nx.DiGraph()
    SG.add_nodes_from(sequence)

    x=gra.subgraph(sequence)

    for i in range(0,len(sequence)):
        for p in range(i, len(sequence)):
            if (sequence[i],sequence[p]) in x.edges():
                SG.add_edge(sequence[i],sequence[p], weight= x[sequence[i]][sequence[p]]['weight'])
    #print(' compute edges seconds', time.time() - start_time)
    return SG
    

def somme(arcs):
    l = list(arcs.edges())
    sum=0
    for tup in l:
        sum=sum + arcs[tup[0]][tup[1]]['weight']
    return sum

def utility_sum(gra,sequence):
    return (somme(compute_edges(gra,sequence)))

# For the conditional variant of OMEGA
def noeuds(E,S):   # E is a set of tuples (l), S is a list of items (nodes)

    a, b = zip(*E)
    S =[ i for i in S ]
    l = list(a)+list(b)
    y= list(set(l))
    z= list(set(y)-set(S))

    return z


def noeuds1(E):
    a,b = zip(*E)
    l = list(a)+list(b)
    y= list(set(l))
    return y

def Union(E,e):
    return E.union(e)

def Union_List(lst1, lst2): 
    final_list = list(set(lst1) | set(lst2)) 
    return len(final_list)

###### REORDER

def REORDER(gra,structure,E,S):  # Compute sequence of items from a set of  edges E according to a graph gra
    #np.random.seed(123)  # to have for every call of the function the same topological structure
    # works in  k.log(k) |A|=k

    A = noeuds(E,S)
    #start_time = time.time()
    b= sorted(A, key=lambda x: structure.index(x))
    #print(' REORDER seconds', time.time() - start_time)

    return b


def REORDER1(gra,structure,E):  # Compute sequence of items from a set of  edges E according to a graph gra
    #np.random.seed(123)  # to have for every call of the function the same topological structure
    # works in  k.log(k) |A|=k

    A = noeuds1(E)
    #start_time = time.time()
    b= sorted(A, key=lambda x: structure.index(x))
    #print(' REORDER seconds', time.time() - start_time)

    return b

# OMEGA
def OMEGA(gra,structure,  k,S):

    edges = set()
    arcs = set(gra.edges()) # set of (edges) tuples [ (0,1) , (1,2),....   ]
    #print(len(arcs))
    # dif = [x for x in arcs if x not in edges]
    dif = arcs-edges
    C=[]    #  implementing the set C and while condition
    for e in dif:
        #print(Union( edges,[e]))

        if len(noeuds(Union( edges,[e]) ,S)) <= k:
            C.append(e)
    #        utilities.append (utility(gra, REORDER(gra,Union(edges,[e]))))


    while C :
        maximum = -1   # searching in C the edge that will maximizes the utility function
        for i in C:
            value  = utility_sum(gra, REORDER1(gra,structure, Union(edges,[i]) ) )
            #print('utility', value)
            if maximum < value:
                maximum= value
                tup= i # the edge to add with the maximum utility
        
        edges.add(tup)

#print(edges)
        if len(noeuds(edges,S )) ==k:
                return  REORDER( gra,structure, edges,S  )

        dif = arcs-edges
       
   #  implementing the set C and while condition
        C=set()
        

        for e in dif:

            if len(noeuds(Union( edges,[e]),S )) <= k:
               C.add(e)

#print(C)
    return  REORDER( gra,structure, edges,S  )


def Precision_k(gra,structure, test_user_sequence, k):
    
    user_id, previous, left = divide_test(test_user_sequence)
    reco = OMEGA(gra,structure,k, previous)
    reco =[str(i) for i in reco]
    
    print(reco)
    intersection = list(set(reco) & set(left) )
    #print(intersection)
    return (len (intersection)/k)



# Our Graph Creation for a sequence (  one user  )
def Create_OUR(test_user_sequence,z):

    transitions = pk.load(open('final_matrix2', 'rb'))
    user_id, previous, left = divide_test(test_user_sequence)
    candidate_items = list(set(movies).difference(previous))
    #candidate_items = left
    gra = nx.DiGraph()
    for j in previous:
        gra.add_node(j)

    for i in candidate_items:

        gra.add_node(i)
        gra.add_edge(i,i)
        index_i = movie_ids[i]
        gra[i][i]['weight'] = transitions[index_i, index_i]

    last_items = previous[-z:]
    for item in last_items:
        index_last = movie_ids[item]

        for i in candidate_items:
            index_item = movie_ids[i]
            gra.add_edge(item,i)
            gra[item][i]['weight'] = transitions[ index_last,index_item]

    return gra


def worker( argument, results):
    
    G= nx.DiGraph()

    G = Create_OUR(argument,4)

    #print(G.edges(data=True))

    toto = nx.DiGraph()

    toto = G.copy()

    toto.remove_edges_from(nx.selfloop_edges(toto,data=True))  # toto

    structure = list(nx.topological_sort(toto))  # toto
    
     
    results.append( Precision_k( G,structure,argument,8))


random.seed(123)
# In the paper, they used 500 sequences at random as a test set
# Split sequences into training and testing

def make_train_test (sequences, size):

    list_shuffled = list_sequences.copy()
    #random.shuffle(list_shuffled)
    train_data = list_shuffled[size:]
    test_data =list_shuffled[:size]
    return train_data, test_data



##########################

def proba(sequences, list_items,topic_dict):
    number_of_items = len(list_items)
    matrice = np.zeros(shape=(number_of_items,number_of_items))  # to store number of occurences with respect to the threshold l
    matrice1 = np.zeros(shape=(number_of_items,number_of_items)) # to store number of occurences

    # Version of the probas without the parameter l
    for seq in sequences:
         items = seq[1:]
         for i in range(len(items)-1):
            for j in range(i+1, len(items)):
                
                index_i = list_items[items[i]]
                
                index_j = list_items[items[j]]
                matrice[index_i][index_j] += Union_List(topic_dict[items[i]],topic_dict[items[j]])
    
    return matrice

#Counting the occurance of an item in all the lists we have
def frequency (train, list_items,topic_dict):
    number_of_items = len(list_items)
    list_count = [0]* number_of_items

    for sequence in train:
        items = sequence[1:]
        for item in items:
          topic=0
          if item in list_items:
              index_item = list_items.index(item)
              topic=len(topic_dict[item]) 
              list_count[index_item]+=topic

    for i in range(len(list_count)):
        if list_count[i]<10:
            list_count[i]=0

    return list_count

# Dividing the occurance of each item by the number of lists we have in our set of sequences
def empirical_frequency_items(train, list_items,topic_dict):  # train: training set of sequences , items list of item ids

    number_of_items = len(list_items)
    list_count = [0]* number_of_items

    for sequence in train:
        items = sequence[1:]
        for item in items:
          topic=0
          if item in list_items:
            index_item = list_items.index(item)
            topic=len(topic_dict[item]) 
            list_count[index_item]+=1

    for i in range(len(list_count)):
        if list_count[i]<10:
            list_count[i]=0

    return [x/len(train) for x in list_count]


def Transition_matrix (list_sequences, movies):

    matrice= pk.load(open ('proba_matrix_without_l', 'rb'))
    # first we run Proba function to get the proba_matrix_without_l, we store it and loaod it for faster tests

    final_matrix = np.zeros(shape= (len(movies), len(movies)))

    frequencies = frequency(list_sequences, movies,topic_dict)
    emp_freq = empirical_frequency_items(list_sequences, movies,topic_dict)
    for i in range(len(movies)):
        for j in range(len(movies)):
            if i == j:
                final_matrix[i,i]= emp_freq[i]
                
            else :
                if frequencies[i] ==0:
                    final_matrix[i,j]= 0
                else:
                    final_matrix[i,j]= matrice[i,j]/frequencies[i]
    return final_matrix


#####################








##########################################
#dividing the data into test and train

train_data, test_data = make_train_test(list_sequences,150)

m = proba(train_data, movie_ids,topic_dict)
print('part2')
pk.dump(m, open('proba_matrix_without_l','wb'))

print('start')
final_matrix = Transition_matrix(train_data, movies)
pk.dump(final_matrix , open('final_matrix2', 'wb')) 

start_time = time.time()
manager = multiprocessing.Manager()
results = manager.list()

jobs=[]
for i in range(150):
        p = multiprocessing.Process(target = worker, args=(test_data[i], results))
        jobs.append(p)
        p.start()

for proc in jobs:
    proc.join()
print ('sum(results)/len(results)',sum(results)/len(results))
print('whole', time.time() - start_time)

In [ ]:
#loading th results obtained from Omega Framework
data=open('Omega_results_ds1.txt','r')

data_2=data.readlines()
results_sequences=[]
for i in range(150):
  line=data_2[i]
  line_1=ast.literal_eval(line)
  line_2=[i] + line_1
  line_3=line_2
  results_sequences.append(line_3)

In [ ]:
len(results_sequences)

In [ ]:
#taking corresponding sequence from the input to be used for final result
test_sequence=list_sequences[:150]

In [ ]:
#For final sequence combine the previous part of input sequence with sequence obtained from omega framework
final_sequence=[]
for i in range(len(test_sequence)):
  list_1=test_sequence[i]
  list_2=list_1[1:8]
  #print(list_2)
  list_3=results_sequences[i]
  list_4=list_3[1:]
  #print(list_4)
  final_sequence.append(list_2 + list_4) 

In [ ]:
len(final_sequence)

In [ ]:
#saving the final results obtained
with open('test_results_ds1.txt', 'w') as f:
    for item in final_sequence:
        f.write("%s\n" % item)